In [1]:
import numpy as np
import cv2
import glob
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
#%matplotlib inline

output_folder = './output_images/'
non_vehicle_folder = './data/non-vehicles/'
vehicle_folder = './data/vehicles/'

In [2]:
def saveImage(img, file_name=None, file_name_extension=None, isRGB=False):
    if file_name!=None:
        # Slice out the filename from the path
        new_file_name = file_name[max(file_name.rfind('\\'), file_name.rfind('/')) + 1:file_name.rfind('.')]
        # If the filename should be extended, add the extension
        if file_name_extension!=None:
            new_file_name += file_name_extension
        # Save the image to a file
        if isRGB:
            cv2.imwrite(output_folder + new_file_name + '.png', cv2.cvtColor(img, cv2.COLOR_RGB2BGR))
        else:
            cv2.imwrite(output_folder + new_file_name + '.png', img)

In [3]:
# Returns some characteristics of the dataset 
def data_look(car_list, notcar_list):
    data_dict = {}
    # Define a key in data_dict "n_cars" and store the number of car images
    data_dict["n_cars"] = len(car_list)
    # Define a key "n_notcars" and store the number of notcar images
    data_dict["n_notcars"] = len(notcar_list)
    # Read in a test image, either car or notcar
    example_img = mpimg.imread(car_list[0])
    # Define a key "image_shape" and store the test image shape 3-tuple
    data_dict["image_shape"] = example_img.shape
    # Define a key "data_type" and store the data type of the test image.
    data_dict["data_type"] = example_img.dtype
    # Return data_dict
    return data_dict

In [4]:
from skimage.feature import hog

def get_hog_features(img, orient, pix_per_cell, cell_per_block, vis=False, feature_vec=False, transform_sqrt=False):
    if vis == True:
        # Use skimage.hog() to get both features and a visualization
        features, hog_image = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                                  cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=transform_sqrt,
                                  visualise=True, feature_vector=feature_vec)
        return features, hog_image
    else:
        # Use skimage.hog() to get features only
        features = hog(img, orientations=orient, pixels_per_cell=(pix_per_cell, pix_per_cell),
                       cells_per_block=(cell_per_block, cell_per_block), transform_sqrt=transform_sqrt,
                       visualise=False, feature_vector=feature_vec)
        return features

In [5]:
# Compute color histogram features from a BGR image
def bin_spatial(img, size=(32, 32)):
    # Use cv2.resize().ravel() to create the feature vector
    features = cv2.resize(img, size).ravel()
    # Return the feature vector
    return features

In [6]:
# Create a feature list of the color channels
def color_hist(img, channel_list_str, nbins, bins_range, file_name=None):
    
    if (len(img.shape)==2) or (img.shape[2]==1):
        channel1_hist = np.histogram(img, bins=nbins, range=bins_range)
    else:
        # Compute the histogram of the color channels separately
        channel1_hist = np.histogram(img[:,:,0], bins=nbins, range=bins_range)
        channel2_hist = np.histogram(img[:,:,1], bins=nbins, range=bins_range)
        channel3_hist = np.histogram(img[:,:,2], bins=nbins, range=bins_range)
    
    if ((file_name!=None) & (len(img.shape)==3)):
        # Store an image of the histogram
        
        # Generate bin centers
        bin_edges = channel1_hist[1]
        bin_centers = (bin_edges[1:] + bin_edges[0:len(bin_edges)-1]) / 2
        
        # Plot a figure with all three bar charts
        fig = plt.figure(figsize=(21,3))
        plt.subplot(131)
        plt.bar(bin_centers, channel1_hist[0])
        plt.xlim(0, 256)
        plt.title(channel_list_str[0] + ' Channel Histogram')
        plt.subplot(132)
        plt.bar(bin_centers, channel2_hist[0])
        plt.xlim(0, 256)
        plt.title(channel_list_str[1] + ' Channel Histogram')
        plt.subplot(133)
        plt.bar(bin_centers, channel3_hist[0])
        plt.xlim(0, 256)
        plt.title(channel_list_str[2] + ' Channel Histogram')
        
        # Slice out the filename from the path
        new_file_name = file_name[max(file_name.rfind('\\'), file_name.rfind('/')) + 1:file_name.rfind('.')]
        # Add the filename extension
        new_file_name += ('_01_' + channel_list_str[0][0] + channel_list_str[1][0] + channel_list_str[2][0] + '_hist')
        
        # Store the image
        fig.savefig(output_folder + new_file_name + '.png')
    
    # Concatenate the histograms into a single feature vector
    if (len(img.shape)==2) or (img.shape[2]==1):
        hist_features = channel1_hist[0]
    else:
        hist_features = np.concatenate((channel1_hist[0], channel2_hist[0], channel3_hist[0]))
    # Return the feature vector
    return hist_features

In [7]:
def extract_features(img, spatial_size, hist_bins, hist_range, channel_list_str, file_name=None):
            
    # Apply bin_spatial() to get spatial color features
    spatial_features = bin_spatial(img, size=spatial_size)
    # Apply color_hist() also with a color space option now
    hist_features = color_hist(img, channel_list_str=channel_list_str, nbins=hist_bins,
                               bins_range=hist_range, file_name=file_name)
    
    # Return feature vector
    return np.concatenate((spatial_features, hist_features))

In [8]:
from mpl_toolkits.mplot3d import Axes3D

def plot3d(pixels, colors_rgb, axis_labels=list("RGB"), axis_limits=[(0, 255), (0, 255), (0, 255)]):
    """Plot pixels in 3D."""

    # Create figure and 3D axes
    fig = plt.figure(figsize=(8, 8))
    ax = Axes3D(fig)

    # Set axis limits
    ax.set_xlim(*axis_limits[0])
    ax.set_ylim(*axis_limits[1])
    ax.set_zlim(*axis_limits[2])

    # Set axis labels and sizes
    ax.tick_params(axis='both', which='major', labelsize=14, pad=8)
    ax.set_xlabel(axis_labels[0], fontsize=16, labelpad=16)
    ax.set_ylabel(axis_labels[1], fontsize=16, labelpad=16)
    ax.set_zlabel(axis_labels[2], fontsize=16, labelpad=16)

    # Plot pixel values with colors given in colors_rgb
    ax.scatter(
        pixels[:, :, 0].ravel(),
        pixels[:, :, 1].ravel(),
        pixels[:, :, 2].ravel(),
        c=colors_rgb.reshape((-1, 3)), edgecolors='none')

    return ax  # return Axes3D object for further manipulation

In [9]:
from sklearn.preprocessing import StandardScaler

def scaleData(X):
    # Create an array stack of feature vectors
    #X = np.vstack((car_features, notcar_features)).astype(np.float64)                        
    # Fit a per-column scaler
    X_scaler = StandardScaler().fit(X)
    # Apply the scaler to X
    scaled_X = X_scaler.transform(X)
    return scaled_X, X_scaler

In [10]:
# Properties for color feature extraction
#color_space = 'BGR'
#color_space = 'HSV'
#color_space = 'LUV'
color_space = 'HLS'
#color_space = 'YUV'
#color_space = 'YCrCb'
spatial_size=(16, 16)
hist_bins=256
hist_range=(0, 256)

# Properties for hog feature extraction
color_channels = [0,1,2,3] # 0,1,2 for the color channel of the color space, 3 for gray scaled image
orient = 9
pix_per_cell = 8
cell_per_block = 2
transform_sqrt = False

store_image_size = (720,720)

def generateImageFeatures(img, file_name=None, display_number_of_features=False,
                          include_color_features=True, include_hog_features=True):
    
    # Convert image to new color space (if specified)
    if color_space != 'BGR':
        if color_space == 'RGB':
            channel_list_str = ['Red', 'Green', 'Blue']
            feature_image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        elif color_space == 'HSV':
            channel_list_str = ['Hue', 'Saturation', 'Value']
            feature_image = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
        elif color_space == 'LUV':
            channel_list_str = ['L', 'U', 'V']
            feature_image = cv2.cvtColor(img, cv2.COLOR_BGR2LUV)
        elif color_space == 'HLS':
            channel_list_str = ['Hue', 'Lightness', 'Saturation']
            feature_image = cv2.cvtColor(img, cv2.COLOR_BGR2HLS)
        elif color_space == 'YUV':
            channel_list_str = ['Y', 'U', 'V']
            feature_image = cv2.cvtColor(img, cv2.COLOR_BGR2YUV)
        elif color_space == 'YCrCb':
            channel_list_str = ['Y', 'Cr', 'Cb']
            feature_image = cv2.cvtColor(img, cv2.COLOR_BGR2YCrCb)
        else:
            print('WARNING: Color space is not defined in the extract_feature function!')
    else:
        channel_list_str = ['Blue', 'Green', 'Red']
        feature_image = np.copy(img)
    
    # Get a list of color features
    color_features = extract_features(feature_image, spatial_size=spatial_size, hist_bins=hist_bins, hist_range=hist_range,
                                      channel_list_str=channel_list_str, file_name=file_name)
    
    hog_features = []
    for channel in color_channels:
        # Create an image of one color
        if channel==3:
            hog_img = cv2.cvtColor(feature_image, cv2.COLOR_BGR2GRAY)
            channel_list_str.append('Gray')
        else:
            hog_img = feature_image[:,:,channel]
        
        if file_name!=None:
            h_features, hog_image = get_hog_features(hog_img, orient=orient, pix_per_cell=pix_per_cell,
                                                        cell_per_block=cell_per_block, vis=True, feature_vec=False,
                                                        transform_sqrt=transform_sqrt)
            hog_image = cv2.resize(hog_image, store_image_size, interpolation=cv2.INTER_NEAREST)
            saveImage(hog_image, file_name=file_name, file_name_extension='_03_hog_' + channel_list_str[channel], isRGB=False)
        else:
            h_features = get_hog_features(hog_img, orient=orient, pix_per_cell=pix_per_cell, cell_per_block=cell_per_block,
                                            vis=False, feature_vec=False, transform_sqrt=transform_sqrt)
        hog_features = np.concatenate((hog_features, h_features.ravel()))
        
    # store the images to use them in the writeup
    if file_name!=None:
        resized_image = cv2.resize(img, store_image_size, interpolation=cv2.INTER_NEAREST)
        saveImage(resized_image, file_name=file_name, file_name_extension=None, isRGB=False)
        for channel in color_channels:
            if channel==3:
                store_image = cv2.cvtColor(resized_image, cv2.COLOR_BGR2GRAY)
            else:
                store_image = resized_image[:,:,channel]
            saveImage(store_image, file_name=file_name,
                        file_name_extension='_02_color_channel_' + channel_list_str[channel], isRGB=False)
    
    # Deside what to return
    if (include_color_features==True) & (include_hog_features==True):
        # Concatinate the color and hog features
        image_features = np.concatenate((color_features, hog_features))
    elif include_color_features==True:
        # Just return color features
        image_features = color_features
    elif include_hog_features==True:
        # Just return hog features
        image_features = hog_features
    else:
        # Senseless state! Return an empty list!
        image_features = []
    
    if display_number_of_features:
        print('Number of extracted color features: ' + str(len(color_features)))
        print('Number of extracted hog features: ' + str(len(hog_features)))
        print('Total Number of extracted data features: ' + str(len(image_features)))
        
    return image_features

In [11]:
# Read the vehicle and non-vehicle images
vehicle_images = glob.glob(vehicle_folder + '/*/*.png')
non_vehicle_images = glob.glob(non_vehicle_folder + '/*/*.png')

# Display basic information about the data
data_info = data_look(vehicle_images, non_vehicle_images)
print(data_info)

{'image_shape': (64, 64, 3), 'n_cars': 8822, 'n_notcars': 9003, 'data_type': dtype('float32')}


In [12]:
# Gerarate the training and testing data
include_color_features=True
include_hog_features=True

image_features_list = []
for fname in np.concatenate((vehicle_images, non_vehicle_images)):
    img = cv2.imread(fname)
    image_features = generateImageFeatures(img, file_name=None, display_number_of_features=False,
                                           include_color_features=include_color_features,
                                           include_hog_features=include_hog_features)
    image_features_list.append(image_features)

# Scale the data and get the scaler to scale the other images
X, X_scaler = scaleData(image_features_list)
y = np.hstack((np.ones(len(vehicle_images)), np.zeros(len(non_vehicle_images))))

print('Total number of examples: ' + str(len(X)))
print('Total number of features: ' + str(len(X[0])))

Total number of examples: 17825
Total number of features: 8592


In [13]:
from sklearn.model_selection import train_test_split

# Split the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [14]:
from sklearn.svm import LinearSVC
from sklearn.model_selection import GridSearchCV

# Try several parameter values for C
parameters = {'C':[0.1, 0.3, 1]}
svc = LinearSVC(random_state=42)
clf = GridSearchCV(svc, parameters)
clf.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=LinearSVC(C=1.0, class_weight=None, dual=True, fit_intercept=True,
     intercept_scaling=1, loss='squared_hinge', max_iter=1000,
     multi_class='ovr', penalty='l2', random_state=42, tol=0.0001,
     verbose=0),
       fit_params={}, iid=True, n_jobs=1, param_grid={'C': [0.1, 0.3, 1]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [15]:
print('Best parameter for C is: ' + str(clf.best_params_['C']))

Best parameter for C is: 0.1


In [16]:
# Use a linear SVC
svc = LinearSVC(C=clf.best_params_['C'], random_state=42)
# Train the SVC
svc.fit(X_train, y_train)

print('Training accuracy of the LinearSVC = ', svc.score(X_train, y_train))
print('Test accuracy of the LinearSVC = ', svc.score(X_test, y_test))

Training accuracy of the LinearSVC =  1.0
Test accuracy of the LinearSVC =  0.993548387097


In [17]:
# Make a prediction for a single image
def predictImage(img, file_name=None):
    # Since the training data consists of 64x64 images, resize the image to the same format
    if img.shape[0:2]!=(64,64):
        img = cv2.resize(img, (64, 64), interpolation=cv2.INTER_NEAREST)
    # Generate the features for the image
    image_features = generateImageFeatures(img, file_name=file_name, display_number_of_features=False,
                                           include_color_features=include_color_features,
                                           include_hog_features=include_hog_features)
    # Scale the features
    image_features = image_features.reshape(1, -1)
    image_features = X_scaler.transform(image_features)
    # Return the prediction
    return svc.predict(image_features)

In [18]:
# Here is your draw_boxes function from the previous exercise
def draw_boxes(img, bboxes, color=(0, 0, 255), thick=6):
    # Make a copy of the image
    imcopy = np.copy(img)
    # Iterate through the bounding boxes
    for bbox in bboxes:
        # Draw a rectangle given bbox coordinates
        cv2.rectangle(imcopy, bbox[0], bbox[1], color, thick)
    # Return the image copy with boxes drawn
    return imcopy

In [19]:
# Define a function that takes an image,
# start and stop positions in both x and y, 
# window size (x and y dimensions),  
# and overlap fraction (for both x and y)
def slide_window(img, x_start_stop=[None, None], y_start_stop=[None, None], 
                    xy_window=(64, 64), xy_overlap=(0.5, 0.5)):
    # If x and/or y start/stop positions not defined, set to image size
    if x_start_stop[0] == None:
        x_start_stop[0] = 0
    if x_start_stop[1] == None:
        x_start_stop[1] = img.shape[1]
    if y_start_stop[0] == None:
        y_start_stop[0] = 0
    if y_start_stop[1] == None:
        y_start_stop[1] = img.shape[0]
    # Compute the span of the region to be searched
    xspan = x_start_stop[1] - x_start_stop[0]
    yspan = y_start_stop[1] - y_start_stop[0]
    # Compute the number of pixels per step in x/y
    nx_pix_per_step = np.int(xy_window[0]*(1 - xy_overlap[0]))
    ny_pix_per_step = np.int(xy_window[1]*(1 - xy_overlap[1]))
    # Compute the number of windows in x/y
    nx_buffer = np.int(xy_window[0]*(xy_overlap[0]))
    ny_buffer = np.int(xy_window[1]*(xy_overlap[1]))
    nx_windows = np.int((xspan-nx_buffer)/nx_pix_per_step)
    ny_windows = np.int((yspan-ny_buffer)/ny_pix_per_step)
    # Initialize a list to append window positions to
    window_list = []
    # Loop through finding x and y window positions
    for ys in range(ny_windows):
        for xs in range(nx_windows):
            # Calculate window position
            startx = xs*nx_pix_per_step + x_start_stop[0]
            endx = startx + xy_window[0]
            starty = ys*ny_pix_per_step + y_start_stop[0]
            endy = starty + xy_window[1]
            # Append window position to list
            window_list.append(((startx, starty), (endx, endy)))
    # Return the list of windows
    return window_list

In [20]:
def add_heat(heatmap, bbox_list):
    # Iterate through list of bboxes
    for box in bbox_list:
        # Add += 1 for all pixels inside each bbox
        # Assuming each "box" takes the form ((x1, y1), (x2, y2))
        heatmap[box[0][1]:box[1][1], box[0][0]:box[1][0]] += 1

    # Return updated heatmap
    return heatmap

In [21]:
def apply_threshold(heatmap, threshold):
    # Zero out pixels below the threshold
    heatmap[heatmap <= threshold] = 0
    # Return thresholded map
    return heatmap

In [22]:
def draw_labeled_bboxes(img, labels):
    # Iterate through all detected cars
    for car_number in range(1, labels[1]+1):
        # Find pixels with each car_number label value
        nonzero = (labels[0] == car_number).nonzero()
        # Identify x and y values of those pixels
        nonzeroy = np.array(nonzero[0])
        nonzerox = np.array(nonzero[1])
        # Define a bounding box based on min/max x and y
        bbox = ((np.min(nonzerox), np.min(nonzeroy)), (np.max(nonzerox), np.max(nonzeroy)))
        # Draw the box on the image
        cv2.rectangle(img, bbox[0], bbox[1], (0,0,255), 6)
    # Return the image
    return img

In [23]:
from scipy.ndimage.measurements import label
heatmap_threshold = 5

global_counter = 0
round_buffer = [[],[],[],[],[]]
round_buffer_size = len(round_buffer)

# Run through the test images and find the lane lines
def detectVehicles(img, file_name=None):
    # Convert to BGR because CV2 uses BRG as default
    local_img = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)
    
    # Create an empty list to receive positive detection windows
    on_windows = []
    search_area = [(400,480),(400,526),(400,558),(400,656)]
    dimensions = [64,96,126,256]
    for i in range(len(dimensions)):
        d = dimensions[i]
        ystart = search_area[i][0]
        ystop = search_area[i][1]
        windows = slide_window(local_img, x_start_stop=[None, None], y_start_stop=[ystart, ystop],
                                   xy_window=(d, d), xy_overlap=(0.75, 0.75))
        
        # Iterate over all windows in the list
        for window in windows:
            sliced_img = local_img[window[0][1]:window[1][1], window[0][0]:window[1][0]]
            prediction = predictImage(sliced_img, file_name=None)
            # If positive (prediction == 1) then save the window
            if prediction == 1:
                on_windows.append(window)
    
    heatmap = np.zeros_like(img[:,:,0])
    if file_name!=None:
        window_img = np.copy(img)
        for box in on_windows:
            window_img = cv2.rectangle(window_img,box[0],box[1],(0,0,255),6)
        saveImage(window_img, file_name=file_name, file_name_extension='_02_boxed_image', isRGB=True)
        heatmap = add_heat(heatmap, on_windows)
        heatmap = apply_threshold(heatmap, heatmap_threshold)
        if np.max(heatmap)>0:
            heatmap *= (int(255/np.max(heatmap)))
        saveImage(heatmap, file_name=fname, file_name_extension='_04_heatmap', isRGB=False)
        labels = label(heatmap)
        labeled_image = draw_labeled_bboxes(np.copy(img), labels)
        saveImage(labeled_image, file_name=fname, file_name_extension='_04_labeled_heatmap', isRGB=True)
    else:
        global global_counter
        global round_buffer
        buffer_pos = global_counter % round_buffer_size
        round_buffer[buffer_pos] = on_windows
        for boxes in round_buffer:
            heatmap = add_heat(heatmap, boxes)
        heatmap = apply_threshold(heatmap, heatmap_threshold)
        labels = label(heatmap)
        labeled_image = draw_labeled_bboxes(np.copy(img), labels)
        global_counter+=1
        
    return labeled_image

In [24]:
# Run the test images and produce some output images
images = glob.glob('./test_images/*.jpg')
for fname in images:
    labeld_image = detectVehicles(cv2.cvtColor(cv2.imread(fname), cv2.COLOR_BGR2RGB), file_name=fname)

In [25]:
# Import MoviePy
from moviepy.editor import VideoFileClip

# Read the images of the video and write the processed images to a new wideo file
video_output = output_folder + 'project_video.mp4'
clip1 = VideoFileClip("project_video.mp4")
video_clip = clip1.fl_image(detectVehicles)
%time video_clip.write_videofile(video_output, audio=False)

[MoviePy] >>>> Building video c:/output_images/project_video.mp4
[MoviePy] Writing video c:/output_images/project_video.mp4


100%|█████████████████████████████████████████████████████████████████████████████▉| 1260/1261 [26:09<00:01,  1.28s/it]


[MoviePy] Done.
[MoviePy] >>>> Video ready: c:/output_images/project_video.mp4 

Wall time: 26min 9s
